In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.2 MB/s 
     |████████████████████████████████| 596 kB 44.9 MB/s 
     |████████████████████████████████| 101 kB 10.7 MB/s 
     |████████████████████████████████| 6.6 MB 32.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import os
import pickle

import pandas as pd
import numpy as np

import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [4]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP_training_course/project/text_classification/ # your path

/content/drive/MyDrive/Colab Notebooks/NLP_training_course/project/text_classification


In [ ]:
# !python preprocess.py

# if to_run:
#   !git clone https://github.com/AntonMu/TrainYourOwnYOLO /content/drive/MyDrive/socar_hackathon/kkh/yolo/TrainYourOwnYOLO
#   %cd /content/drive/MyDrive/socar_hackathon/kkh/yolo/TrainYourOwnYOLO
#   to_run = False
#   # !wandb login

Downloading: 100% 226k/226k [00:00<00:00, 25.7MB/s]
Downloading: 100% 28.0/28.0 [00:00<00:00, 24.0kB/s]
Downloading: 100% 570/570 [00:00<00:00, 489kB/s]
it will take some times...
make id file finished!


In [5]:
from dataset import *
from trainer_copy import Trainer

In [6]:
open_file = open(os.path.join("./data/train_pos.pkl"), "rb")
train_pos = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join("./data/train_neg.pkl"), "rb")
train_neg = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join("./data/dev_pos.pkl"), "rb")
dev_pos = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join("./data/dev_neg.pkl"), "rb")
dev_neg = pickle.load(open_file)
open_file.close()

open_file = open(os.path.join("./data/test.pkl"), "rb")
test = pickle.load(open_file)
open_file.close()

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_dataset = SentimentDataset(tokenizer, train_pos, train_neg)
dev_dataset = SentimentDataset(tokenizer, dev_pos, dev_neg)
test_dataset = SentimentTestDataset(tokenizer, test)

train_batch_size=128
eval_batch_size=64
test_batch_size = 32

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size,
                                           shuffle=True, collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)

dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=eval_batch_size,
                                         shuffle=False, collate_fn=collate_fn_style,
                                         num_workers=2)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                          shuffle=False, collate_fn=collate_fn_style_test,
                                          num_workers=2)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
random_seed=42
np.random.seed(random_seed)
torch.manual_seed(random_seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)
learning_rate = 5e-5

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [9]:
trainer = Trainer(train_loader=train_loader, dev_loader=dev_loader, test_loader=test_loader, model=model, lr=learning_rate, optimizer=AdamW, device=device)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [10]:
trainer.train(train_epoch=1)

Eval:  98%|█████████▊| 62/63 [00:03<00:00, 17.58it/s]
                                                     

Acc for model which have lower valid loss:  0.97075


Eval:  98%|█████████▊| 62/63 [00:03<00:00, 17.69it/s]
                                                     

Acc for model which have lower valid loss:  0.9745


Eval:  98%|█████████▊| 62/63 [00:03<00:00, 17.58it/s]
                                                     

...

Eval:  98%|█████████▊| 62/63 [00:03<00:00, 17.44it/s]
                                                     

Acc for model which have lower valid loss:  0.97725


Eval:  98%|█████████▊| 62/63 [00:03<00:00, 17.19it/s]
                                                     

Acc for model which have lower valid loss:  0.98


Epoch 0: 100%|██████████| 3463/3463 [17:34<00:00,  3.28batch/s, loss=0.00723]


In [11]:
predictions=trainer.test()


Test:  97%|█████████▋| 31/32 [00:01<00:00, 24.63it/s]
                                                     

In [14]:
test_df = pd.read_csv('./data/test_no_label.csv')
test_df['Category'] = predictions
test_df.to_csv('./submission/submission_test.csv', index=False)

In [15]:
test_df

,Id,Category
0,it 's a whole new experience and new flavors e...,1
1,so disappointing from an old favorite .,0
2,it is the most authentic thai in the valley .,1
3,do not sign a lease with these people .,0
4,i was nervous and she made me feel so comforta...,1
...,...,...
995,the food here is delicious .,1
996,we 'll certainly be back !,1
997,the building itself looks abandoned .,0
998,if i could give zero stars i def would .,0
